Plano detalhado 
Para transformar o vídeo inicial no vídeo final, seguem as etapas e técnicas que serão utilizadas:

Inversão da Imagem a Cada 20 Segundos, criação de uma função para inverter a imagem no eixo vertical a cada 20 segundos utilizando o clip.fl.
Diminuição Gradativa do Volume do Áudio, criação de uma função decrease para diminuir o volume do áudio gradativamente ao longo do tempo, com os últimos 10 segundos ficando em silêncio usando audio.fl.
Reordenamento dos Segmentos do Vídeo para fazer o corte do segmento entre 60 e 80 segundos e reordenar os segmentos, colocando o segmento cortado no final do vídeo.

In [1]:
import numpy as np
import moviepy.editor as mpe

In [2]:
video = mpe.VideoFileClip("video_trabalho.mp4")
audio = video.audio
duracao_total = video.duration
# Função para inverter a imagem no eixo vertical
def inverter(frame):
    return np.flip(frame, axis=0)  
# Função para processar o vídeo e inverter a cada 20 segundos
def processando(clip):
    def inverter_20(get_frame, t):
        if int(t) // 20 % 2 == 1:
            frame = get_frame(t)
            return inverter(frame)
        else:
            return get_frame(t)
    return clip.fl(inverter_20)
# Aplica a inversão ao vídeo
video_invertido = processando(video)

In [3]:
# Função para diminuir o volume do áudio gradativamente
def decrease(get_audio_frame, t):
    frame = get_audio_frame(t)
    if isinstance(t, np.ndarray):
        mask = t >= duracao_total - 10
        fator = np.where(mask, 0.0, 1.0 - t / (duracao_total - 10))
    else:
        if t >= duracao_total - 10:
            fator = 0.0
        else:
            fator = 1.0 - t / (duracao_total - 10)
    if frame.ndim == 1:
        frame *= fator
    else:
        frame[:, 0] *= fator  
        frame[:, 1] *= fator  
    return frame
# Aplica a diminuição gradativa do volume ao áudio
audio_decrease = audio.fl(decrease)
# Combina o vídeo invertido com o áudio com volume ajustado
video_semi = video_invertido.set_audio(audio_decrease)

In [4]:
# Segmentação do vídeo
seg1 = video_semi.subclip(0, 60)
seg2 = video_semi.subclip(60, 80)
seg3 = video_semi.subclip(80)
# Reordena os segmentos, colocando o segmento de 60 a 80 segundos no final
video_final = mpe.concatenate_videoclips([seg1, seg3, seg2])
# Exporta o vídeo final com todas as transformações aplicadas
video_final.write_videofile("video_final.mp4", codec='libx264')

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
